# Nashville Housing Data Cleaning

Descriptions

The first block of code allows us to load the ipython-sql library and connect to the local database database Hashville_Hosing where the table Data_2013_2016 is located.

In [1]:
%load_ext sql
%config SqlMagic.displaycon = False


Please note that column 9 to column to column 19 will be excluded as over half of the data are missing from these columns in the original dataset, so there is no reason to include these columns.

For the purpose of demonstration, we will only clean and transform the first 8 columns of this dataset.

In [2]:
%%sql
IF NOT EXISTS (SELECT * FROM tempdb.sys.objects WHERE name = 'temp_data_cleaning' AND type = 'U')
BEGIN
    SELECT UniqueID, ParcelID, LandUse, PropertyAddress, SaleDate, SalePrice, LegalReference, SoldAsVacant
    INTO #temp_data_cleaning
    FROM Data_2013_2016   
END
COMMIT;

56477 rows affected.


[]

We will run the following code to display the first 10 rows of the table and identify issues in this table.

In [3]:
%%sql
select top(10) *
from #temp_data_cleaning

Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant
2045.0,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09 00:00:00,240000.0,20130412-0036474,No
16918.0,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10 00:00:00,366000.0,20140619-0053768,No
54582.0,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26 00:00:00,435000.0,20160927-0101718,No
43070.0,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",2016-01-29 00:00:00,255000.0,20160129-0008913,No
22714.0,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",2014-10-10 00:00:00,278000.0,20141015-0095255,No
18367.0,007 00 0 151.00,SINGLE FAMILY,"1821 FOX CHASE DR, GOODLETTSVILLE",2014-07-16 00:00:00,267000.0,20140718-0063802,No
19804.0,007 14 0 002.00,SINGLE FAMILY,"2005 SADIE LN, GOODLETTSVILLE",2014-08-28 00:00:00,171000.0,20140903-0080214,No
54583.0,007 14 0 024.00,SINGLE FAMILY,"1917 GRACELAND DR, GOODLETTSVILLE",2016-09-27 00:00:00,262000.0,20161005-0105441,No
36500.0,007 14 0 026.00,SINGLE FAMILY,"1428 SPRINGFIELD HWY, GOODLETTSVILLE",2015-08-14 00:00:00,285000.0,20150819-0083440,No
19805.0,007 14 0 034.00,SINGLE FAMILY,"1420 SPRINGFIELD HWY, GOODLETTSVILLE",2014-08-29 00:00:00,340000.0,20140909-0082348,No


Standardise Date format
Using Alter function to change data type
Then verify the results

In [4]:
%%sql
alter table #temp_data_cleaning
alter column saledate date
commit;

select top(10) saledate
from #temp_data_cleaning


Done.
Done.


saledate
2013-04-09
2014-06-10
2016-09-26
2016-01-29
2014-10-10
2014-07-16
2014-08-28
2016-09-27
2015-08-14
2014-08-29


Check null values

In [5]:
%%sql
select *
from #temp_data_cleaning
where UniqueID is null or ParcelID is null or LandUse is null or PropertyAddress is null or SaleDate is null or SalePrice is null or LegalReference is null or SoldAsVacant is null


Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant
43076.0,025 07 0 031.00,SINGLE FAMILY,None,2016-01-15,179900.0,20160120-0005776,No
39432.0,026 01 0 069.00,VACANT RESIDENTIAL LAND,None,2015-10-23,153000.0,20151028-0109602,No
45290.0,026 05 0 017.00,SINGLE FAMILY,None,2016-03-29,155000.0,20160330-0029941,No
53147.0,026 06 0A 038.00,RESIDENTIAL CONDO,None,2016-08-25,144900.0,20160831-0091567,No
43080.0,033 06 0 041.00,SINGLE FAMILY,None,2016-01-04,170000.0,20160107-0001526,No
45295.0,033 06 0A 002.00,SINGLE FAMILY,None,2016-03-29,210000.0,20160331-0030709,No
48731.0,033 15 0 123.00,SINGLE FAMILY,None,2016-05-05,199900.0,20160506-0045368,No
50927.0,044 05 0 135.00,SINGLE FAMILY,None,2016-06-15,160000.0,20160617-0061987,No
3299.0,052 01 0 296.00,SINGLE FAMILY,None,2013-05-31,79370.0,20130620-0063114,No
40678.0,042 13 0 075.00,SINGLE FAMILY,None,2015-11-30,208000.0,20151209-0123831,No


Populate the propertyAddress column to replace the null values
Rows that have the same parcelID also share the same address

In [6]:
%%sql
update a
set a.PropertyAddress=b.PropertyAddress
from #temp_data_cleaning as a
inner join #temp_data_cleaning as b
on a.parcelID=b.ParcelID
and a.uniqueID!=b.UniqueID
where a.PropertyAddress is null
commit;

select UniqueID, ParcelID, PropertyAddress
from #temp_data_cleaning
where ParcelID='025 07 0 031.00' or ParcelID='026 01 0 069.00'

29 rows affected.
Done.


UniqueID,ParcelID,PropertyAddress
38077.0,025 07 0 031.00,"410 ROSEHILL CT, GOODLETTSVILLE"
43076.0,025 07 0 031.00,"410 ROSEHILL CT, GOODLETTSVILLE"
22721.0,026 01 0 069.00,"141 TWO MILE PIKE, GOODLETTSVILLE"
39432.0,026 01 0 069.00,"141 TWO MILE PIKE, GOODLETTSVILLE"


Find duplicates

In [7]:
%%sql
select a.*
from #temp_data_cleaning as a
join (select UniqueID, count(*) as count
from #temp_data_cleaning
group by UniqueID
having count(*)>1) as b
on a.UniqueID = b.UniqueID


0 rows affected.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant


In [8]:
%%sql
select parcelid, propertyaddress, saledate, saleprice, legalreference, count(*) as duplicateCount
from #temp_data_cleaning
group by parcelid, propertyaddress, saledate, saleprice, legalreference
having count(*) >1

Done.


parcelid,propertyaddress,saledate,saleprice,legalreference,duplicateCount
081 02 0 144.00,"1728 PECAN ST, NASHVILLE",2015-02-02,57000.0,20150205-0010843,2
081 07 0 265.00,"1806 15TH AVE N, NASHVILLE",2015-02-17,65000.0,20150223-0015122,2
081 10 0 313.00,"1626 25TH AVE N, NASHVILLE",2015-02-20,35000.0,20150224-0015904,2
081 11 0 168.00,"1710 DR D B TODD JR BLVD, NASHVILLE",2015-02-13,44500.0,20150218-0013602,2
081 11 0 495.00,"1718 ARTHUR AVE, NASHVILLE",2015-02-09,36500.0,20150210-0012450,2
081 15 0 263.00,"1520 14TH AVE N, NASHVILLE",2015-02-12,55000.0,20150218-0013742,2
081 15 0 472.00,"1818 B SCOVEL ST, NASHVILLE",2015-02-20,35000.0,20150223-0015257,2
090 08 0 191.00,"743 CROLEY DR, NASHVILLE",2015-02-13,169000.0,20150219-0014430,2
090 11 0A 030.00,"515 BASSWOOD AVE, NASHVILLE",2015-02-05,60000.0,20150209-0011960,2
090 12 0 091.00,"501 FOUNDRY DR, NASHVILLE",2015-02-18,208000.0,20150223-0015576,2


Delete duplicates

In [9]:
%%sql
with duplicateCTE as 
(
	select *, ROW_NUMBER() 
	over (partition by parcelid, propertyaddress, saledate, saleprice, legalreference order by uniqueid) as duplicateCount
	from #temp_data_cleaning
)


delete from duplicateCTE
where duplicateCount > 1
commit;


104 rows affected.


[]

Validate

In [10]:
%%sql
select parcelid, propertyaddress, saledate, saleprice, legalreference, count(*) as duplicateCount
from #temp_data_cleaning
group by parcelid, propertyaddress, saledate, saleprice, legalreference
having count(*) >1

0 rows affected.


parcelid,propertyaddress,saledate,saleprice,legalreference,duplicateCount


Breaking property address into 2 different columns using substring

In [11]:
%%sql
alter table #temp_data_cleaning
add PropertyStreet nvarchar(255), PropertyCity nvarchar(255)
commit;

update #temp_data_cleaning
set 
PropertyStreet = 
	substring(PropertyAddress, 
	1, 
	charindex(',', PropertyAddress)-1),
PropertyCity = 
	substring(PropertyAddress, 
	charindex(',', PropertyAddress)+1, 
	len(PropertyAddress))
commit;


Done.
56373 rows affected.


[]

Breaking OwerAddress into 3 different columns

%%sql
alter table #temp_data_cleaning
add OwnerStreet nvarchar(255), OwnerCity nvarchar(255), OwnerState nvarchar(255)
commit;

update #temp_data_cleaning
set 
OwnerStreet = 
	substring(OwnerAddress, 
	1, 
	charindex(',', OwnerAddress)-1),
OwnerCity = 
	substring(OwnerAddress, 
	charindex(',', OwnerAddress)+2, 
	charindex(',', OwnerAddress, charindex(',', OwnerAddress) + 1) -charindex(',', OwnerAddress)-2),
OwnerState = 
	substring(OwnerAddress, 
	charindex(',', OwnerAddress, charindex(',', OwnerAddress) + 1) + 2, 
	len(OwnerAddress) - charindex(',', OwnerAddress, charindex(',', OwnerAddress) + 1) - 1)
	commit;



Verify the result

In [12]:
%%sql
select top(5) PropertyAddress, PropertyStreet, PropertyCity
from #temp_data_cleaning

Done.


PropertyAddress,PropertyStreet,PropertyCity
"1808 FOX CHASE DR, GOODLETTSVILLE",1808 FOX CHASE DR,GOODLETTSVILLE
"1832 FOX CHASE DR, GOODLETTSVILLE",1832 FOX CHASE DR,GOODLETTSVILLE
"1864 FOX CHASE DR, GOODLETTSVILLE",1864 FOX CHASE DR,GOODLETTSVILLE
"1853 FOX CHASE DR, GOODLETTSVILLE",1853 FOX CHASE DR,GOODLETTSVILLE
"1829 FOX CHASE DR, GOODLETTSVILLE",1829 FOX CHASE DR,GOODLETTSVILLE


Check data consistancy

In [13]:
%%sql
select top (10) * from #temp_data_cleaning

Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,PropertyStreet,PropertyCity
2045.0,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09,240000.0,20130412-0036474,No,1808 FOX CHASE DR,GOODLETTSVILLE
16918.0,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10,366000.0,20140619-0053768,No,1832 FOX CHASE DR,GOODLETTSVILLE
54582.0,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26,435000.0,20160927-0101718,No,1864 FOX CHASE DR,GOODLETTSVILLE
43070.0,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",2016-01-29,255000.0,20160129-0008913,No,1853 FOX CHASE DR,GOODLETTSVILLE
22714.0,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",2014-10-10,278000.0,20141015-0095255,No,1829 FOX CHASE DR,GOODLETTSVILLE
18367.0,007 00 0 151.00,SINGLE FAMILY,"1821 FOX CHASE DR, GOODLETTSVILLE",2014-07-16,267000.0,20140718-0063802,No,1821 FOX CHASE DR,GOODLETTSVILLE
19804.0,007 14 0 002.00,SINGLE FAMILY,"2005 SADIE LN, GOODLETTSVILLE",2014-08-28,171000.0,20140903-0080214,No,2005 SADIE LN,GOODLETTSVILLE
54583.0,007 14 0 024.00,SINGLE FAMILY,"1917 GRACELAND DR, GOODLETTSVILLE",2016-09-27,262000.0,20161005-0105441,No,1917 GRACELAND DR,GOODLETTSVILLE
36500.0,007 14 0 026.00,SINGLE FAMILY,"1428 SPRINGFIELD HWY, GOODLETTSVILLE",2015-08-14,285000.0,20150819-0083440,No,1428 SPRINGFIELD HWY,GOODLETTSVILLE
19805.0,007 14 0 034.00,SINGLE FAMILY,"1420 SPRINGFIELD HWY, GOODLETTSVILLE",2014-08-29,340000.0,20140909-0082348,No,1420 SPRINGFIELD HWY,GOODLETTSVILLE


Check ParcellID column
15 length = normal
16 length = rows without data after column 9

In [14]:
%%sql
SELECT LEN(ParcelID) AS ParcelID_Length, COUNT(DISTINCT ParcelID) AS Distinct_Count
FROM #temp_data_cleaning
GROUP BY LEN(ParcelID);

Done.


ParcelID_Length,Distinct_Count
15,23370
16,25189


Check LandUse column

In [15]:
%%sql
Select LandUse, count(*)
from #temp_data_cleaning
group by LandUse
order by LandUse asc

Done.


LandUse,
APARTMENT: LOW RISE (BUILT SINCE 1960),2
CHURCH,33
CLUB/UNION HALL/LODGE,1
CONDO,247
CONDOMINIUM OFC OR OTHER COM CONDO,35
CONVENIENCE MARKET WITHOUT GAS,1
DAY CARE CENTER,2
DORMITORY/BOARDING HOUSE,19
DUPLEX,1372
FOREST,10


Fix inconsistant data and typo
GREENBELT/RESGRRENBELT/RES
VACANT RESIENTIAL LAND

In [16]:
%%sql
update #temp_data_cleaning
set LandUse = 'GREENBELT/RES'
where LandUse like '%GRRENBELT/RES'
commit;

update #temp_data_cleaning
set LandUse = 'VACANT RESIDENTIAL LAND'
where LandUse ='VACANT RESIENTIAL LAND'
commit;


3 rows affected.
3 rows affected.


[]

Validate the result

In [17]:
%%sql
Select LandUse, count(*)
from #temp_data_cleaning
where LandUse = 'GREENBELT/RES' or LandUse like '%GRRENBELT/RES' or LandUse = 'VACANT RESIDENTIAL LAND' or LandUse = 'VACANT RESIENTIAL LAND'
group by LandUse
order by LandUse asc

Done.


LandUse,
GREENBELT/RES,3
VACANT RESIDENTIAL LAND,3543


Check SaleDate
The ISDATE() function checks whether a string is a valid date or not. It returns 1 if the expression is a valid date, otherwise 0
CONVERT(varchar, SaleDate, 121) converts the SaleDate column to a string in the format 'YYYY-MM-DD' (which is style 121)

In [18]:
%%sql
SELECT SaleDate
FROM #temp_data_cleaning
WHERE ISDATE(CONVERT(varchar, SaleDate, 121)) = 0;

0 rows affected.


SaleDate


Check SalePrice format

In [19]:

%%sql
SELECT SalePrice
FROM #temp_data_cleaning
WHERE SalePrice NOT LIKE '%[0-9.0-9]' 

0 rows affected.


SalePrice


Check LegalReference length

In [20]:
%%sql
SELECT LegalReference AS LegalReference_Length, COUNT(DISTINCT ParcelID) AS Distinct_Count
FROM #temp_data_cleaning
GROUP BY LEN(LegalReference);

(pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Column '#temp_data_cleaning.LegalReference' is invalid in the select list because it is not contained in either an aggregate function or the GROUP BY clause. (8120) (SQLExecDirectW)")
[SQL: SELECT LegalReference AS LegalReference_Length, COUNT(DISTINCT ParcelID) AS Distinct_Count
FROM #temp_data_cleaning
GROUP BY LEN(LegalReference);]
(Background on this error at: https://sqlalche.me/e/20/f405)


It seems 16 character is the standard format, however unable to verify, in real world senario this will be report back to the data owner

In [21]:
%%sql
SELECT * 
FROM #temp_data_cleaning
WHERE LEN(LegalReference) !=16

Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,PropertyStreet,PropertyCity
42865.0,128 13 0A 158.00,RESIDENTIAL CONDO,"158 WESTFIELD DR, NASHVILLE",2016-01-28,108000.0,-2016946,No,158 WESTFIELD DR,NASHVILLE
43048.0,059 15 0A 147.00,SINGLE FAMILY,"912 BORDEAUX PL, NASHVILLE",2016-01-04,264757.0,20160112-00003053,No,912 BORDEAUX PL,NASHVILLE
20017.0,083 01 0D 001.00,RESIDENTIAL CONDO,"1118 A SHARPE AVE, NASHVILLE",2014-08-29,419240.0,-2022158,No,1118 A SHARPE AVE,NASHVILLE
46422.0,104 10 0 154.00,VACANT RESIDENTIAL LAND,"3206 OVERLOOK DR, NASHVILLE",2016-04-06,410929.0,20160408-00339999,Yes,3206 OVERLOOK DR,NASHVILLE
48164.0,104 10 0O 009.00,SINGLE FAMILY,"112 RANSOM AVE, NASHVILLE",2016-05-18,839820.0,20160519-00503315,No,112 RANSOM AVE,NASHVILLE
2863.0,104 10 0R 007.00,RESIDENTIAL CONDO,"607 CHESTERFIELD WAY, NASHVILLE",2013-05-17,363320.0,2013520-0050586,No,607 CHESTERFIELD WAY,NASHVILLE
28415.0,062 07 0 001.00,VACANT RESIDENTIAL LAND,"2929 WESTERN HILLS DR, NASHVILLE",2015-03-09,50000.0,20150310 -0020554,No,2929 WESTERN HILLS DR,NASHVILLE
17927.0,104 16 0 287.00,SINGLE FAMILY,"2402 OAKLAND AVE, NASHVILLE",2014-07-30,695150.0,-2020988,No,2402 OAKLAND AVE,NASHVILLE
4743.0,083 14 0 153.00,SINGLE FAMILY,"1801 FATHERLAND ST, NASHVILLE",2013-06-07,449830.0,20130610-00588852,No,1801 FATHERLAND ST,NASHVILLE
43449.0,136 07 0 030.00,SINGLE FAMILY,"117 TIMBER RIDGE DR, NASHVILLE",2016-01-14,160000.0,-2016598,No,117 TIMBER RIDGE DR,NASHVILLE


Check SoldAsVacant

In [22]:
%%sql
Select distinct SoldAsVacant, count(*)
from #temp_data_cleaning
group by SoldAsVacant

Done.


SoldAsVacant,
N,398
Yes,4617
Y,52
No,51306


In [23]:
%%sql
UPDATE #temp_data_cleaning
SET SoldAsVacant =
CASE
    WHEN SoldAsVacant = 'Y' THEN 'Yes'
    WHEN SoldAsVacant = 'N' THEN 'No'
    ELSE SoldAsVacant
END
COMMIT;

56373 rows affected.


[]

Validate

In [24]:
%%sql
Select distinct SoldAsVacant, count(*)
from #temp_data_cleaning
group by SoldAsVacant

Done.


SoldAsVacant,
Yes,4669
No,51704


Check PropertyStreet (use top 10 as the list is too long)

In [25]:
%%sql
SELECT PropertyStreet
FROM #temp_data_cleaning
WHERE PropertyStreet NOT LIKE '[0-9]%[A-Za-z]%'

Done.


PropertyStreet
DICKERSON PIKE
KNIGHT DR
JOCELYN HOLLOW RD
SUNNYBROOK DR
EVERETT DR
EVERETT DR
MONROE ST
MONROE ST
MONROE ST
MONROE ST


Unable to populate street as many ParcelID have different street names and they are all in incorrect format, report to data owner

In [26]:
%%sql

SELECT ParcelID, PropertyStreet	
from #temp_data_cleaning
where ParcelID in 
(SELECT ParcelID
FROM #temp_data_cleaning
WHERE PropertyStreet NOT LIKE '[0-9]%[A-Za-z]%')
order by ParcelID


Done.


ParcelID,PropertyStreet
025 00 0 218.00,DICKERSON PIKE
049 00 0 375.00,KNIGHT DR
062 00 0 262.00,PENNINGTON BEND RD
063 12 0 062.00,HADLEYS BEND BLVD
063 12 0 063.00,HADLEYS BEND BLVD
069 06 0A 001.00,ASHLAND CITY HWY
069 06 0A 002.00,ASHLAND CITY HWY
069 06 0A 003.00,ASHLAND CITY HWY
069 06 0A 004.00,ASHLAND CITY HWY
069 06 0A 005.00,ASHLAND CITY HWY


In [27]:
%%sql
Select distinct PropertyCity, count(*)
from #temp_data_cleaning
group by PropertyCity

Done.


PropertyCity,
OLD HICKORY,1415
WHITES CREEK,97
MOUNT JULIET,180
UNKNOWN,1
JOELTON,11
GOODLETTSVILLE,735
ANTIOCH,6286
BELLEVUE,1
FRANKLIN,1
MADISON,2114


In [28]:
%%sql
SELECT * 
FROM #temp_data_cleaning
where ParcelID in (Select ParcelID
from #temp_data_cleaning
where PropertyCity like '%UNKNOWN%') 

Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,PropertyStreet,PropertyCity
12726.0,093 06 1B 618.00,RESIDENTIAL CONDO,"231 5TH AVE N, NASHVILLE",2014-02-11,255900.0,20140214-0013082,No,231 5TH AVE N,NASHVILLE
46010.0,093 06 1B 618.00,RESIDENTIAL CONDO,"0 5TH AVE N, UNKNOWN",2016-03-31,298000.0,20160404-0031713,No,0 5TH AVE N,UNKNOWN


In [29]:
%%sql
UPDATE #temp_data_cleaning
set PropertyStreet = '231 5TH AVE N', PropertyCity = 'NASHVILLE'
where UniqueID = '46010.0'
commit

1 rows affected.


[]

Validate

In [30]:
%%sql
SELECT * 
FROM #temp_data_cleaning
where ParcelID ='093 06 1B 618.00'

Done.


UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,PropertyStreet,PropertyCity
12726.0,093 06 1B 618.00,RESIDENTIAL CONDO,"231 5TH AVE N, NASHVILLE",2014-02-11,255900.0,20140214-0013082,No,231 5TH AVE N,NASHVILLE
46010.0,093 06 1B 618.00,RESIDENTIAL CONDO,"0 5TH AVE N, UNKNOWN",2016-03-31,298000.0,20160404-0031713,No,231 5TH AVE N,NASHVILLE
